# Import Libraries

In [1]:
# for drive access
from google.colab import drive
drive.mount('/content/drive')

# standard libraries
import numpy as np
import pandas as pd
import pickle
import time

# for audio
import librosa

# for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# for saving the label & features to disk
import pickle

Mounted at /content/drive


# Self Defined Class Methods For Feature Extraction

In [2]:
class Extraction:

  def __init__(self,
               train_df,
               test_df,
               sr=16000,
               n_mfccs=20, #20 is the default n_mfccs from librosa
               n_mels=20, #reduced from 128 default from librosa to 20
               n_chroma=12, #12 is the default n_chroma from librosa
               features=['mfcc'],
               do_normalize=True,
               do_avgpool=True #WARNING: THE KERNEL WILL CRASH IF YOU TRY TO EXTRACT ALL FEATURES WITHOUT AVGPOOL
               ):
    """
    Instantiate the Extraction class and extract the labels and features from train and val DataFrames.

    The extract_features() method is automatically called to extract the labels and features from the given DataFrame.

    Parameters:
      train_df (pd.DataFrame): DataFrame containing training data.
      test_df (pd.DataFrame): DataFrame containing validation data.
      sr (int): Sample rate of the audio files.
      n_mfccs (int): Number of MFCCs to extract.
      n_mels (int): Number of Mel bands to extract.
      n_chroma (int): Number of chroma bins to use.
      features (list): List of features to extract.
        accepted features: 'mfcc', 'chroma', 'rms', 'spectral_centroid', 'melspectrogram', 'continent', 'rating', 'type'.
      normalize (bool): Whether to normalize the features.
      maxpool (bool): Whether to maxpool the features.
    """
    # instantiate class variables
    self.sr = sr
    self.n_mfccs = n_mfccs
    self.n_mels = n_mels
    self.n_chroma = n_chroma
    self.do_normalize = do_normalize
    self.do_avgpool = do_avgpool

    # confirm features have been specified
    assert len(features) != 0, "Must Specify At Least One Feature In The Form Of A List."
    self.features = features

    # confirm features are valid
    self.accepted_feature = ['mfcc', 'chroma', 'rms', 'spectral_centroid', 'melspectrogram', 'continent', 'rating', 'type']
    for each in self.features:
      assert each in self.accepted_feature, f"{each} is not an accepted feature, only 'mfcc', 'chroma', 'rms', 'spectral_centroid', 'melspectrogram', 'continent', 'rating', and 'type' are accpeted features."

    # extract train and val labels and features
    self.train_y, self.train_features = self.extract_features(train_df)
    self.test_y, self.test_features = self.extract_features(test_df)

    # process the features by applying normalization or average pooling
    self.train_features, self.test_features = self.process_features(self.train_features, self.test_features)


  ########################################
  # each of the extract_x() function is callable by the extract_features() function based on the features instantiated
  # each of the features are extracted from librosa and transposed to shape (n_time, n_features)
  ########################################
  def extract_mfcc(self, each):
    return np.transpose(librosa.feature.mfcc(y=np.array(each), sr=self.sr, n_mfcc=self.n_mfccs))

  def extract_chroma(self, each):
    return np.transpose(librosa.feature.chroma_stft(y=np.array(each), sr=self.sr, n_chroma=self.n_chroma))

  def extract_rms(self, each):
    return np.transpose(librosa.feature.rms(y=np.array(each)))

  def extract_spectral_centroid(self, each):
    return np.transpose(librosa.feature.spectral_centroid(y=np.array(each), sr=self.sr))

  def extract_melspectrogram(self, each):
    mel = librosa.feature.melspectrogram(y=np.array(each), sr=self.sr, n_mels=self.n_mels)
    return np.transpose(librosa.power_to_db(mel))


  ########################################
  # normalization function is called by process_features() function for the features that require normalization
  ########################################
  def normalization(self, train_X, test_X, n_time, n_features):
    """
    Normalize the train and val features

    Parameters:
      train_X (npy): Training feature
      test_X (npy): Validation feature
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Normalized training feature
      test_X (npy): Normalized validation feature
    """
    # flatten the np arrays to 1D
    train_X_reshape = train_X.reshape(-1, n_time * n_features)
    test_X_reshape = test_X.reshape(-1, n_time * n_features)

    # use minmaxscaler to normalize the train and val features
    scaler = MinMaxScaler((0,1)).fit(train_X_reshape)
    train_X_scaled = scaler.transform(train_X_reshape)
    test_X_scaled = scaler.transform(test_X_reshape)

    # reshape the features to the original shape (n_time, n_features)
    train_X = train_X_scaled.reshape(-1, n_time, n_features)
    test_X = test_X_scaled.reshape(-1, n_time, n_features)

    return train_X, test_X


  ########################################
  # avgpooling function is called by process_features() function for the features that require average pooling
  ########################################
  def avgpooling(self, train_X, test_X, n_time, n_features):
    """
    Average pooling the train and val features

    Parameters:
      train_X (npy): Training feature
      test_X (npy): Validation feature
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Avgpooled training feature
      test_X (npy): Avgpooled validation feature
    """
    # instantiate the keras layers and model.
    # the model only performs average pooling of inputs
    tf.keras.backend.clear_session()
    input = tf.keras.layers.Input(shape=(n_time, n_features))
    global_avg_pooling = tf.keras.layers.GlobalAveragePooling1D()
    pooled = global_avg_pooling(input)
    pooling_model = tf.keras.models.Model(inputs=input, outputs=pooled)

    # use the instantiated model to avgpool the train and val features
    train_X = pooling_model.predict(train_X)
    test_X = pooling_model.predict(test_X)

    return train_X, test_X


  ########################################
  # process_features function is called when the class is instantiated
  ########################################
  def process_features(self, train_features_dict, test_features_dict):
    """
    Apply normalization or average pooling to train and val features

    Parameters:
      train_features_dict (dict): Dictionary containing the training features
      test_features_dict (dict): Dictionary containing the validation features

    Returns:
      tuple: Tuple containing normalized and/or average pooled training and validation features dictionary
    """
    start_time = time.time()
    display('features processing started ---------->')

    for each in train_features_dict.keys():
      # 'continent', 'rating', and 'type' do not need to be processed
      if each == 'continent' or each == 'rating' or each == 'type':
        pass
      # process numeric features
      else:
        if each == 'mfcc':
          n_features=self.n_mfccs
        elif each == 'chroma':
          n_features=self.n_chroma
        elif each == 'rms' or each == 'spectral_centroid':
          n_features=1
        elif each == 'melspectrogram':
          n_features=self.n_mels

        # index the dictionary to find the value of the feature based on the feature name (key)
        train_feature = train_features_dict[each]
        test_feature = test_features_dict[each]

        # call normalization or avgpooling function to apply normalization or average pooling to features
        if self.do_normalize and self.do_avgpool:
          train_features_dict[each], test_features_dict[each] = self.normalization(train_feature, test_feature, n_time=train_feature.shape[1], n_features=n_features)
          train_features_dict[each], test_features_dict[each] = self.avgpooling(train_features_dict[each], test_features_dict[each], n_time=train_feature.shape[1], n_features=n_features)
        elif self.do_normalize:
          train_features_dict[each], test_features_dict[each] = self.normalization(train_feature, test_feature, n_time=train_feature.shape[1], n_features=n_features)
        elif self.do_avgpool:
          train_features_dict[each], test_features_dict[each] = self.avgpooling(train_feature, test_feature, n_time=train_feature.shape[1], n_features=n_features)
        else:
          train_features_dict[each], test_features_dict[each] = train_features_dict[each], test_features_dict[each]

        display(f'{each} processed')

    end_time = time.time()
    display(f'features processing took {(end_time - start_time)/60:.1f} mins')

    return train_features_dict, test_features_dict


  ########################################
  # extract_features function is called when the class is instantiated
  ########################################
  def extract_features(self, dataframe):
    """
    Extract the label & features from the dataframes

    Parameters:
      dataframe (pd.DataFrame): DataFrame containing audio file information

    Returns:
      tuple: Tuple containing labels and a dictionary of extracted features
    """
    start_time = time.time()
    display(f'feature extraction started ---------->')

    # create empty list and dict to store the labels and features
    y = []
    features_dict = {item: [] for item in self.features}

    # iterate through each row of the dataframe to extract the label and features
    for _, row in dataframe.iterrows():
      label = row['primary_label']
      framed = row['framed']

      for each in framed:
        y.append(label)

        if 'continent' in self.features:
          features_dict['continent'].append(row['continent'])
        if 'rating' in self.features:
          features_dict['rating'].append(row['rating'])
        if 'type' in self.features:
          features_dict['type'].append(row['type'])

        # dynatically call the extract_x function to extract the listed features
        for feature in self.features:
          extract = f"extract_{feature}"
          if hasattr(self, extract) and callable(func := getattr(self, extract)):
            features_dict[feature].append(func(each))

    # cast lists to np arrays
    for each in features_dict.keys():
      features_dict[each] = np.array(features_dict[each])
    y = np.array(y)

    end_time = time.time()
    display(f'features extraction took {(end_time - start_time)/60:.1f} mins')

    return y, features_dict


# Load train and test framed audio data

In [3]:
with open('/content/drive/MyDrive/Projects/train_val_csv_pkl/train_df_8_sec.pkl', 'rb') as file:
  train_df = pickle.load(file)

train_df.head()

,primary_label,type,latitude,longitude,rating,filename,duration,country,continent,filename_npy,set,framed
668,barswa,blank,-33.2377,26.9574,5.0,barswa/XC713037.ogg,58.331000,ZA,AFRICA,barswa/XC713037.npy,train,"((tf.Tensor(-1.3069126e-05, shape=(), dtype=fl..."
247,eaywag1,call,54.1191,13.3760,3.5,eaywag1/XC658750.ogg,49.680000,DE,EUROPE,eaywag1/XC658750.npy,train,"((tf.Tensor(-1.1132361e-06, shape=(), dtype=fl..."
620,barswa,song,35.0573,34.0009,5.0,barswa/XC405617.ogg,114.207375,CY,ASIA,barswa/XC405617.npy,train,"((tf.Tensor(3.5154517e-06, shape=(), dtype=flo..."
86,comsan,call,43.5352,-1.4475,4.5,comsan/XC580687.ogg,59.820000,FR,EUROPE,comsan/XC580687.npy,train,"((tf.Tensor(-7.335485e-06, shape=(), dtype=flo..."
450,eaywag1,song,43.5118,3.8367,3.0,eaywag1/XC567939.ogg,10.031000,FR,EUROPE,eaywag1/XC567939.npy,train,"((tf.Tensor(7.8262474e-07, shape=(), dtype=flo..."


In [4]:
with open('/content/drive/MyDrive/Projects/test_csv_pkl/test_df_8_sec.pkl', 'rb') as file:
  test_df = pickle.load(file)

test_df.head()

,primary_label,type,latitude,longitude,rating,filename,duration,country,continent,filename_npy,framed
0,barswa,blank,53.2509,5.5980,4.0,barswa/XC721711.ogg,19.069375,NL,EUROPE,barswa/XC721711.npy,"((tf.Tensor(1.0793006e-05, shape=(), dtype=flo..."
1,comsan,call,48.8306,2.1999,4.0,comsan/XC496602.ogg,28.995938,FR,EUROPE,comsan/XC496602.npy,"((tf.Tensor(-1.02154445e-05, shape=(), dtype=f..."
3,eaywag1,call,37.1357,-7.6138,4.5,eaywag1/XC481360.ogg,151.944000,PT,EUROPE,eaywag1/XC481360.npy,"((tf.Tensor(-1.7517013e-06, shape=(), dtype=fl..."
4,barswa,blank,19.3551,-99.0467,5.0,barswa/XC698512.ogg,18.703688,MX,AMERICAS,barswa/XC698512.npy,"((tf.Tensor(-2.801371e-06, shape=(), dtype=flo..."
5,comsan,call,56.0851,47.2602,5.0,comsan/XC492456.ogg,44.512687,RU,EUROPE,comsan/XC492456.npy,"((tf.Tensor(-1.2049219e-05, shape=(), dtype=fl..."


# Extract features

In [5]:
features_list = ['type','continent','rating']

In [6]:
%%time

features = Extraction(train_df,
                      test_df,
                      features=features_list,
                      do_normalize=False,
                      do_avgpool=False)

'feature extraction started ---------->'

'features extraction took 0.0 mins'

'feature extraction started ---------->'

'features extraction took 0.0 mins'

'features processing started ---------->'

'features processing took 0.0 mins'

CPU times: user 2.88 s, sys: 6.72 ms, total: 2.88 s
Wall time: 2.96 s


In [7]:
train_y = features.train_y

display(train_y.shape)
display(train_y[:5])

(4763,)

array(['barswa', 'barswa', 'barswa', 'barswa', 'barswa'], dtype='<U7')

In [8]:
train_features = features.train_features

for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'type'

(4763,)

'blank'

'continent'

(4763,)

'AFRICA'

'rating'

(4763,)

5.0

In [9]:
test_y = features.test_y
test_y = np.array(test_y)

display(test_y.shape)
display(test_y[:5])

(3402,)

array(['barswa', 'barswa', 'barswa', 'comsan', 'comsan'], dtype='<U7')

In [10]:
test_features = features.test_features

for key in test_features.keys():
  display(key)
  display(test_features[key].shape)
  display(test_features[key][0])

'type'

(3402,)

'blank'

'continent'

(3402,)

'EUROPE'

'rating'

(3402,)

4.0

# Encode Classes

In [11]:
label_encoder = LabelEncoder().fit(train_y)
train_y = label_encoder.transform(train_y)
test_y = label_encoder.transform(test_y)

classes = list(label_encoder.inverse_transform([0,1,2]))
classes

['barswa', 'comsan', 'eaywag1']

In [12]:
display(len(train_y))
display(train_y[:10])

display(len(test_y))
display(test_y[:10])

4763

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

3402

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 2])

# Put the labels and features to one large dataframe

In [13]:
train_features['label'] = train_y
train_features

{'type': array(['blank', 'blank', 'blank', ..., 'blank', 'blank', 'blank'],
       dtype='<U5'),
 'continent': array(['AFRICA', 'AFRICA', 'AFRICA', ..., 'EUROPE', 'EUROPE', 'EUROPE'],
       dtype='<U8'),
 'rating': array([5., 5., 5., ..., 5., 5., 5.]),
 'label': array([0, 0, 0, ..., 2, 2, 2])}

In [14]:
test_features['label'] = test_y
test_features

{'type': array(['blank', 'blank', 'blank', ..., 'call', 'call', 'call'],
       dtype='<U5'),
 'continent': array(['EUROPE', 'EUROPE', 'EUROPE', ..., 'EUROPE', 'EUROPE', 'EUROPE'],
       dtype='<U8'),
 'rating': array([4., 4., 4., ..., 3., 3., 3.]),
 'label': array([0, 0, 0, ..., 0, 0, 0])}

In [16]:
merged_dict = {'test': test_features}

merged_dict

{'test': {'type': array(['blank', 'blank', 'blank', ..., 'call', 'call', 'call'],
        dtype='<U5'),
  'continent': array(['EUROPE', 'EUROPE', 'EUROPE', ..., 'EUROPE', 'EUROPE', 'EUROPE'],
        dtype='<U8'),
  'rating': array([4., 4., 4., ..., 3., 3., 3.]),
  'label': array([0, 0, 0, ..., 0, 0, 0])}}

# save the merged dict with labels and features to pkl

In [17]:
with open('/content/drive/MyDrive/Projects/test_features_pkl/test_8_sec_non_audio_features.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

In [18]:
with open('/content/drive/MyDrive/Projects/test_features_pkl/test_8_sec_non_audio_features.pkl', 'rb') as file:
  merged_df = pickle.load(file)

In [19]:
assert merged_dict.keys() == merged_df.keys()

for top_level_key in merged_dict.keys():
  assert merged_dict[top_level_key].keys() == merged_df[top_level_key].keys()

  for bottom_level_key in merged_dict[top_level_key].keys():
    assert merged_dict[top_level_key][bottom_level_key].shape == merged_df[top_level_key][bottom_level_key].shape
    assert np.array_equal(merged_dict[top_level_key][bottom_level_key], merged_df[top_level_key][bottom_level_key])